## EDA (Análise exploratória de dados)

In [ ]:
# Para variáveis contínuas:
sns.pairplot(tb_insu.select_dtypes(include = 'number'))

In [ ]:
# Para variáveis discretas:
fig, ax = plt.subplots(1,3, figsize = (12,4))
sns.boxplot(data = tb_insu, x = 'sex', y = 'expenses', ax=ax[0])
sns.boxplot(data = tb_insu, x = 'smoker', y = 'expenses', ax=ax[1])
sns.boxplot(data = tb_insu, x = 'region', y = 'expenses', ax=ax[2])

In [ ]:
# tratamento de dados discretos para entrada de modelos de machine learning
pd.get_dummies(tb_insu['obese'], prefix = 'obese') # prefixo muda automaticamente o nome da coluna incluindo categorias

In [ ]:
# sempre utilizar o dataset normalizado
from sklearn.preprocessing import StandardScaler
X = tb_insu[['obese_smoker', 'age', 'bmi_smoker', 'smoker_yes']]
scaler = StandardScaler()
scaler.fit(X)
scaler.transform(X)

Y = tb_insu['expenses']
X_norm = scaler.transform(X)

## Linear Regression

#### Bibliotecas

In [1]:
# biblioteca padrão para machine Learning
from sklearn.linear_model import LinearRegression
# biblioteca padrão para modelos estatíticos
import statsmodels.api as sm

#### Statsmodel

In [ ]:
def simular_dado(media_X, desvpad_X, 
                 desvpad_E, A, B, 
                 samples):
    x = np.random.normal(loc = media_X, scale = desvpad_X, size = samples)
    E = np.random.normal(loc = 0, scale = desvpad_E, size = samples)
    y = B + A * x + E
    return pd.DataFrame({'x' : x, 'y' : y})

In [ ]:
def estimar_lm(data):
    X = sm.add_constant(data['x'])
    Y = data['y']
    modelo = sm.OLS(Y, X)
    lm_fit = modelo.fit()
    return lm_fit

teste_1 = simular_dado(media_X=0,
                       desvpad_X=1,
                       desvpad_E=10,
                       A=1,
                       B=0,
                       samples=100)
teste1_fit = estimar_lm(teste_1)
teste1_fit.summary()

RMSE = np.sqrt(np.mean([residual**2 for residual in teste1_fit.resid]))

#### Sklearn

In [3]:
modelo = LinearRegression()
X = teste_1[['x']]
Y = teste_1['y']
modelo.fit(X, Y)
modelo.coef_
modelo.intercept_
teste_1['pred'] = modelo.predict(X)

sns.scatterplot(data=teste_1, x = 'x', y = 'y')
sns.lineplot(data=teste_1, x = 'x', y = 'pred', color = 'red')

#### Multiple Linear Regression

In [ ]:
# utilizado no caso de regressões com mais de uma variáve de entrada (o que é a maioria dos nossos problemas)
X = sm.add_constant(tb_insu[['obese_smoker', 'age', 'smoker_yes']])
Y = tb_insu['expenses']
modelo = sm.OLS(Y, X)
lm_fit = modelo.fit()
lm_fit.summary()

In [ ]:
tb_insu['lm_pred'] = lm_fit.predict()
sns.pairplot(tb_insu[['expenses', 'lm_pred', 'age', 'bmi']])

In [ ]:
# comparando erros de regressões Linear_regression
test_data["pred_trend_linear"] = trend_fit.predict(test_data[["t_dias"]])
test_data["pred_trend_spline"] = spline_fit.predict(test_data[spl_names])
rmse_lin = np.sqrt(
    mean_squared_error(test_data["per_canceladas"], test_data["pred_trend_linear"])
)
rmse_spl = np.sqrt(
    mean_squared_error(test_data["per_canceladas"], test_data["pred_trend_spline"])
)
print(f"Trend Lin.: {round(rmse_lin, 2)}")
print(f"Trend Spline.: {round(rmse_spl, 2)}")

### Séries temporais

In [ ]:
# A forma mais comum de visualizar dados de série temporal é através de **lineplots** onde o tempo é colocado no eixo X e as variáveis a serem analisadas no eixo Y.
fig, ax = plt.subplots(3, 1, figsize=(15, 8))
sns.lineplot(data=tb_resort_ts, x="arrival_dt", y="num_reservas", ax=ax[0])
sns.lineplot(data=tb_resort_ts, x="arrival_dt", y="num_canceladas", ax=ax[1])
sns.lineplot(data=tb_resort_ts, x="arrival_dt", y="per_canceladas", ax=ax[2])
ax[0].set_xlabel("")
ax[1].set_xlabel("")
fig.suptitle("Evolução de Reservas por Data de Chegada", y=0.95)

In [ ]:
# divisão em train and test
tb_resort_ts["arrival_dt"].describe()
def split_hotel_ts(hotel_data):
    train_data = hotel_data[hotel_data["arrival_dt"] < "2017-08-01"].copy()
    test_data = hotel_data[hotel_data["arrival_dt"] >= "2017-08-01"].copy()
    return train_data, test_data

In [2]:
# Decomposição TCSI

#A forma mais tradicional de se analisar um série temporal é através da decomposição TCSI:

# T - Componente de Tendência (Trend):** representação da evolução a longo prazo da série (*a taxa de cancelamento está crescendo ou caindo?*)
# C - Componente Ciclico (Cyclic)** representação da evolução ciclica não periódicas (*depois de altas de cancelamento tenho baixas?*)
# S - Componente Sazonal (Seasonal)** representação da peridiocidade sazonal (ciclo regular) (*temos mais cancelamento para reservas em junho? ou para reservas de segunda-feira? ou da última semana do mês?*)
# I - Componente de Ruído (Irregular/Noise)** o que não é representado pelos três componentes acima.

In [ ]:
# tendência linear:
train_data, test_data = split_hotel_ts(tb_resort_ts)
trend_fit = lm.LinearRegression()
trend_fit.fit(train_data[["t_dias"]], train_data["per_canceladas"])
tb_resort_ts["pred_trend_linear"] = trend_fit.predict(tb_resort_ts[["t_dias"]])

In [ ]:
# Componente ciclico